In [3]:
from utils.datapreparation import *
train_image,train_label, test_image, test_label, val_image, val_label=LRP_Penobscot(shape=(992,192), stridetrain=(128,64), strideval=(128,64), stridetest=(128,64))


In [1]:
import tensorflow as tf
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print('TensorFlow:', tf.__version__)


2024-10-31 02:37:16.695773: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-31 02:37:16.721329: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 02:37:16.721358: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 02:37:16.722414: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 02:37:16.728178: I tensorflow/core/platform/cpu_feature_guar

TensorFlow: 2.15.0


In [2]:
from models.attention import Attention_unet
from models.unet import Unet
from models.unet3plus import Unet_3plus
from models.bridgenet import FlexibleBridgeNet
from models.efficientNetB1 import EfficientNetB1
from network.CFPNetM import CFPNetM
from network.ENet import ENet
from network.ESPNet import ESPNet
from network.ICNet import ICNet


In [3]:
""" filters=[]
for i in range(0,int(6)):
      filters.append(2**(4+i))
model = Unet_3plus(tam_entrada=(992, 192, 1), n_filters=filters, classes=6,kernel_size=11,dropout_rate=0) """


' filters=[]\nfor i in range(0,int(6)):\n      filters.append(2**(4+i))\nmodel = Unet_3plus(tam_entrada=(992, 192, 1), n_filters=filters, classes=6,kernel_size=11,dropout_rate=0) '

In [3]:
# model=ICNet(992, 192, 1, 6)
model = EfficientNetB1(992,192, 1, 6)

2024-10-31 02:37:54.197918: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-31 02:37:54.225869: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-31 02:37:54.228649: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [15]:
""" import lwbna_unet as unet
model = unet.LWBNAUnet(
    n_classes=6, 
    filters=64, 
    depth=4, 
    midblock_steps=4, 
    dropout_rate=0.3, 
    name="my_unet"
)
model.build(input_shape=(16,992,192,1)) """


' import lwbna_unet as unet\nmodel = unet.LWBNAUnet(\n    n_classes=6, \n    filters=64, \n    depth=4, \n    midblock_steps=4, \n    dropout_rate=0.3, \n    name="my_unet"\n)\nmodel.build(input_shape=(16,992,192,1)) '

In [ ]:
#model = FlexibleBridgeNet(input_size=(992, 192, 1),up_down_times=5, Y_channels=1, kernel_size=11,
                              kernels_all=[16, 32, 64, 128, 256, 512], conv2act_repeat=2, res_case=0,
                              res_number=0)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 992, 192, 1)]     0         
                                                                 
 efficientnetb1 (Functional  (None, 31, 6, 1280)       6574659   
 )                                                               
                                                                 
 conv2d_transpose (Conv2DTr  (None, 62, 12, 256)       2949376   
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 124, 24, 128)      295040    
 Transpose)                                                      
                                                                 
 conv2d_transpose_2 (Conv2D  (None, 248, 48, 64)       73792     
 Transpose)                                                  

In [5]:
#model = tf.keras.applications.ResNet50()

forward_pass = tf.function(
    model.call,
    input_signature=[tf.TensorSpec(shape=(1,992,192,1))])

graph_info = profile(forward_pass.get_concrete_function().graph,
                        options=ProfileOptionBuilder.float_operation())
# The //2 is necessary since `profile` counts multiply and accumulate
# as two flops, here we report the total number of multiply accumulate ops
flops = graph_info.total_float_ops // 2
print('Flops: {:,}'.format(flops))

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Flops: 3,467,596,648
=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              1
-min_occurrence             0
-step                       -1
-order_by                   float_ops
-account_type_regexes       .*
-start_name_regexes         .*
-trim_name_regexes          
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     float_ops
-output                     stdout:

==================Model Analysis Report======================


Doc:
scope: The